
                               Projektseminar Business Analytics - Sommersemester 2022          
                                      Test des künstlichen neuronalen Netzes              
                      Nutzung eines vergleichbaren Datensatzes mit dem das KNN traniert wurde 
                               zusätzlich kann ein bestimmtes Datum eingegeben werden      



In [19]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import pandas as pd
from sklearn.metrics import f1_score
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

In [20]:
def read_data(stadt, wetter, column, typ):
    '''
    Einlesen der Daten

    Parameters
    ----------
    stadt : str
        Stadt für die eine Vorhersage getroffen werden soll.
    wetter : str
        Gibt ein ob Wetterdaten miteinbezogen werden sollen oder nicht.
    column : str
        Gibt an welche Spalte der Daten die wahre Vorhersage ist.
    typ : str
        Gibt an ob die Kategorie oder die Unfallanzahl vorhergesagt werden soll.

    Returns
    -------
    x : DataFrame
        Enthält die InputDaten, auf Grundlage derer eine Voraussage getroffen wird.
    y : array
        Enthält die tatsächlichen Werte zum späteren Vergleich zu Vorhersage.

    '''
    
    path="../Daten/Output/" + 'Trainingsdaten_' + str(typ) + '_' + str(stadt) +".csv"
    df_data = pd.read_csv(path)
            
    y = df_data.values[:,4:5]
    x = df_data.drop(columns = [column])
    if wetter == "Nein":
        x = x.values[:,:4]


    return x, y

In [21]:
def load_model(wetter, model_typ, stadt_knn):
    '''
    Einlesen und laden des trainierten Modells.

    Parameters
    ----------
    wetter : str
        Gibt an ob das Modell geladen werden soll, was mit Wetterdaten 
        trainiert wurde oder das ohne.
    model_typ : str
        Gibt an welches Modell geladen werden soll hinsichtlich der Vorhersage.
        (Unfallanzahl oder Kategorie)
    stadt_knn : str
        Gibt an welches Modell bezüglich der Stadt geladen werden soll.
        wenn stadt = 'Mainz' dann 'Wiesbaden'
        wenn stadt = 'Wiesbaden' dann 'Mainz'

    Returns
    -------
    model : keras.engine.sequential.Sequential
        Traniertes Modell.

    '''
    
    if wetter == 'Nein':
        model_path = '../Daten/Output/model_' + str(model_typ) + '_' + str(stadt_knn)
    else:
        model_path = '../Daten/Output/model_' + str(model_typ) + '_' + str(stadt_knn) + '_weather'
        
    model = keras.models.load_model(model_path)
    
    return model

In [22]:
def model_test(model, x):
    '''
    Testen des trainierten KNNs anhand der Daten x.

    Parameters
    ----------
    model : keras.engine.sequential.Sequential
        Das trainierte Modell.
    x : DataFrame
        Testdaten für das trainierte KNN.
       
    Returns
    -------

    prediction : DataFrame
        Vorhersage für den Output auf Grundlage der Daten von x.
        
    predictions_with_input : DataFrame
        Vorhersage mit Inputdaten zusammengeführt.

    '''

    
    prediction = pd.DataFrame(model.predict(x)) 
    
    predictions_with_input = pd.DataFrame(np.c_[x, prediction])
    
    return prediction, predictions_with_input


In [23]:
def data_hist(y, predictions):
    '''
    Daten zum Vergleich der Vorhersagewerte und den tatsächlichen Werten.

    Parameters
    ----------
    y : DataFrame
        Tatsächliche Werte.
    predictions : DataFrame
        Vorhergesagte Werte.

    Returns
    -------
    diff : array
        Differenzen zwischen der Vorhersage und den tatsächlichen Werten.

    '''
    predictions = np.array(predictions)
    
    diff = [(np.argmax(predictions[i]))-y[i] for i in range(len(y))]
    
    diff = np.array(diff)
    
    
    unique, counts = np.unique(diff, return_counts=True)
    result_dict = dict(zip(unique, counts))
        
    for u, c in zip(unique, counts):
        print(f'# {u} : {c}')
    print('------------------------------------------------')   
    correct = result_dict.get(0.0, 0)
    total = np.sum(counts)
        
    print(f'Es wird eine Accuracy von {correct/total:.2f} erreicht')
    print('------------------------------------------------')
    return diff


In [24]:
def histogram(diff, stadt, stadt_knn):
    '''
    Histogramm zur veranschulichung der Differenz zwischen 
    tatsächlichem und vorhergesagtem Wert

    Parameters
    ----------
    diff : array
        Enthält die Differenzen von Vorhersage und tatsächlichem Wert.

    Returns
    -------
    None.

    '''
    
    fig2, ax2 = plt.subplots(1,1,sharex=True, figsize=(17, 10), dpi=300)
    ax2.set_xticks([-5,-4, -3, -2, -1, 0, 1,2,3,4])
    ax2.tick_params(axis='both', which='major', labelsize=18)
    ax2.tick_params(axis='both', which='minor', labelsize=18)
    plt.title(f'Vorhersage für {stadt} auf Grundlage des KNN {stadt_knn}',fontsize=30)
    plt.xlabel('Differenz zur tatsächlichen Unfallanzahl', fontsize=30)
    plt.ylabel('Anzahl', fontsize=30)
    ax2.hist(diff, bins=[-5,-4.5,-4,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4,4.5])
    plt.show()
    
    return


In [25]:
def f1(y_test, predictions):
    
    max_unfaelle = int(y_test.max())
    predictions = np.array(predictions)
    
    f1_macro = f1_score(y_test, np.argmax(predictions, axis = -1), average = 'macro')
    print(f'f1_macro : {f1_macro}')
    print('------------------------------------------------')
    f1_label = [f1_score(y_test, np.argmax(predictions, axis = -1), labels= [i], pos_label=1, average='macro') for i in range(0,max_unfaelle+1)]
    
    for label,f1 in enumerate(f1_label):
        print(f'Der f1-Score der Klasse {label} beträgt: {f1}.')


    return f1_macro, f1_label



In [26]:
style = {'description_width': 'initial'}

typ = widgets.Dropdown(
    options=[('Anzahl der Unfälle', 'U'), ('Unfallkategorie', 'K')],
    description= 'Soll die Unfallanzahl oder die Kategorie des Unfalls vorhergesagt werden?',
    value = 'U',
    style =style,
    layout = Layout(width='80%')
)

date = widgets.Dropdown(
    options=[('Ja'), ('Nein')],
    description= 'Möchten Sie ein genaues Datum eingeben?',
    value = 'Ja',
    style =style,
    layout = Layout(width='80%')
)
stadt = widgets.Dropdown(
    options=['Mainz', 'Wiesbaden'],
    description='In welche Stadt möchten Sie fahren ?',
    value = 'Mainz',
    style =style,
    layout = Layout(width='50%')
)
stadt2 = widgets.Dropdown(
    options=['Mainz', 'Wiesbaden'], 
    description='Für welche Stadt soll das KNN eine Vorhersage machen?',
    value = 'Mainz',
    style =style,
    layout = Layout(width='50%')
)
    
wetter = widgets.Dropdown(
    options=[('Ja'), ('Nein')],
    description= 'Möchten Sie zusätzlich zum Zeitpunkt auch Wetterdaten mit eingeben?',
    value = 'Ja',
    style =style,
    layout = Layout(width='80%')
)

wetter2 = widgets.Dropdown(
    options=[('Ja'), ('Nein')],
    description= 'Möchten Sie das KNN mit Wetterdaten testen?',
    value = 'Ja',
    style =style,
    layout = Layout(width='80%')
)

year = widgets.IntSlider(
    value=1950,
    min=1950,
    max=2025,
    step=1,
    description='Geben Sie ein Jahr an:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style =style,
    layout = Layout(width='80%')
)
    
month = widgets.IntSlider(
    value=6,
    min=1,
    max=12,
    step=1,
    description='Geben Sie einen Monat an:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style =style,
    layout = Layout(width='80%')
)
    
day = widgets.Dropdown(
    options=[('Montag',1),('Dienstag',2),('Mittwoch',3),('Donnerstag',4),('Freitag',5),('Samstag',6),('Sonntag',7)],
    value=1,
    description='Geben Sie einen Wochentag an:',
    style =style,
    layout = Layout(width='50%')
)
hour = widgets.IntSlider(
    value=12,
    min=1,
    max=24,
    step=1,
    description='Geben Sie eine Stunde an:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style =style,
    layout = Layout(width='80%')
)


nied = widgets.IntSlider(
    value=0,
    min=0,
    max=30,
    step=1,
    description='Wie viel Niederschlag in mm wird erwartet?',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style =style,
    layout = Layout(width='80%')
)
    
temp = widgets.IntSlider(
    value=15,
    min=-20,
    max=40,
    step=1,
    description='Wie hoch ist die Lufttempreatur in Grad Celsius?',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style =style,
    layout = Layout(width='80%')
)

In [27]:
def prediction(date,stadt,wetter,year,month,day,hour,nied,temp,wetter2, stadt2):
    
    column = '# Unfaelle'
    model_typ = 'num'
    typ = 'AnzahlUnfaelle'
   
    if date == 'Ja' :
        if stadt == 'Mainz':
            stadt_knn = 'Mainz'
        else:
            stadt_knn = 'Wiesbaden'

        if wetter == 'Ja':
            x = np.array([year, month, day, hour, nied, temp])
        else:
            x = np.array([year, month, day, hour])

        x = np.reshape(x, (1,len(x)))

        model = load_model(wetter, model_typ, stadt_knn)
        prediction, prediction_with_input = model_test(model,x)
        

    else:    
        
        wetter = wetter2
        stadt = stadt2

        if stadt == 'Mainz':
            stadt_knn = 'Wiesbaden'
        else: 
            stadt_knn = 'Mainz'

        x,y = read_data(stadt,wetter,column,typ)
        model = load_model(wetter, model_typ, stadt_knn)
        prediction, prediction_with_input = model_test(model,x)
        diff = data_hist(y, prediction)
        histogram(diff, stadt, stadt_knn)
        f1_macro, f1_label = f1(y,prediction)
        
    return prediction
        
interact(prediction,date = date,stadt = stadt,wetter = wetter,year = year,month = month,day=day,hour = hour,nied = nied,temp = temp, wetter2 = wetter2, stadt2=stadt2)




interactive(children=(Dropdown(description='Möchten Sie ein genaues Datum eingeben?', layout=Layout(width='80%…

<function __main__.prediction(date, stadt, wetter, year, month, day, hour, nied, temp, wetter2, stadt2)>